In [1]:
import sys, os
sys.path.append('../')
from config.basic import ConfigBasic
from networks.util import prepare_model
import numpy as np
import torch
from torch import nn
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

# Dataset
cfg = ConfigBasic()
cfg.dataset = 'aiims'
cfg.setting = 'D'
cfg.fold = 4

cfg.logscale = False
cfg.set_dataset()
cfg.tau = 1

# Model
cfg.model = 'GOL'
cfg.backbone = 'vgg16v2norm'
cfg.metric = 'L2'
cfg.k = np.arange(2, 50, 2)
cfg.epochs = 40
cfg.scheduler = 'cosine'
cfg.lr_decay_epochs = [100, 200, 300]
cfg.period = 3

cfg.margin = 0.25
cfg.ref_mode = 'flex'
cfg.ref_point_num = 10  # 60 Fold1, 58 Fold0 setting D // 56 setting c // 58 setting B // 55 setting A
cfg.drct_wieght = 1
cfg.start_norm = True
cfg.learning_rate = 0.0001

# Log
#cfg.wandb = False
#cfg.experiment_name = 'EXP_NAME'
#cfg.save_folder = f'../../RESULT_FOLDER_NAME/{cfg.dataset}/setting{cfg.setting}/{cfg.experiment_name}/PREFIX_{cfg.margin}_tau{cfg.tau}_F{cfg.fold}_{cfg.model}_{cfg.backbone}_{get_current_time()}'
#make_dir(cfg.save_folder)

cfg.n_gpu = torch.cuda.device_count()
cfg.num_workers = 1


class NeuralNetwork(nn.Module):
    def __init__(self, split, ind=None) -> None:
            super().__init__()
            self.ind = ind
            self.model = prepare_model(cfg)
            self.model.load_state_dict(torch.load("models/split_"+split+"/0/model.tar"))
            
    
    def forward(self, x):
        x = self.model(x)
        #dist_mat = torch.matmul(x[0], self.model.ref_points[self.ind].T)
        dist_mat = torch.matmul(x, self.model.ref_points.T)
        #return torch.stack([dist_mat,])
        return dist_mat
    
class NeuralNetwork_pred(nn.Module):
    def __init__(self, split, ind=None) -> None:
            super().__init__()
            self.ind = ind
            self.model = prepare_model(cfg)
            self.model.load_state_dict(torch.load("models/split_"+split+"/0/model.tar"))
            
    
    def forward(self, x):
        x = self.model(x)
        #dist_mat = torch.matmul(x[0], self.model.ref_points[self.ind].T)
        #dist_mat = torch.matmul(x, self.model.ref_points.T)
        #return torch.stack([dist_mat,])
        return x

In [2]:
import json
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from captum.attr import GuidedGradCam , GuidedBackprop, LRP, LayerGradCam, LayerAttribution

from captum.attr._utils.lrp_rules import EpsilonRule, PropagationRule
from captum.attr import visualization as viz


def get_box_params( index, src="/home/raman/Work/Dataset/boxes.json"):
    data = json.load(open(src , 'r'))
    box = data[str(index)]
    # h, w = data["imageHeight"], data["imageWidth"]
    return box

def homographic_transformation(file, index,side = 300):
    
    box = get_box_params(index)
    
    # Define the four corners of the rotated box
    rotated_box = np.array(box, dtype=np.float32)

    # Define the four corners of the target box
    target_box = np.array([(0, 0), (side, 0), (side, side), (0, side)], dtype=np.float32)

    # Calculate the homography matrix
    H, _ = cv2.findHomography(rotated_box, target_box)

    # Load an image
    image = cv2.imread(file)

    # Apply the homography transformation to the image
    transformed_image = cv2.warpPerspective(image, H, (side, side))
    transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_BGRA2RGB)
    #import ipdb
    #ipdb.set_trace()
    return Image.fromarray(transformed_image).convert('RGB')

In [3]:
import pickle
with (open("outputs_gol.pickle", "rb")) as openfile:
    outputs = pickle.load(openfile)
with (open("labels_gol.pickle", "rb")) as openfile:
    labels = pickle.load(openfile)

In [4]:
from torchvision import transforms
from pathlib import Path
import pandas as pd

preds = {}
pred_labels = {}
cls = 1
outputs = [i[0] for i in outputs]
labels = np.array(labels)
predictions = []
def get_visulaization(exp):
    fld = "_".join(exp.split("_")[2:])
    save_addr = "/home/raman/Work/Code/drilling/GOL/train/grad_cam_class_"+str(cls)+"/"
    Path(save_addr).mkdir(parents=True, exist_ok=True)
    df=pd.read_csv('/home/raman/Work/Dataset/new_score_split.csv')
    
    for fold in range(1):
        model = model = NeuralNetwork_pred(str(fold), cls)
        model.eval()
        # val_dataset = AgeDB(data_dir=str(fold), df=None, img_size=224, group=fld, split='val')
        # val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False,
        #                     num_workers=32, pin_memory=True, drop_last=False)
        #model.model.lnorm.rule = EpsilonRule()
        #print(model.model.lnorm.rule)
        target_layer = model.model.encoder.features[-1]
        #cap_vis = LayerGradCam(model, target_layer)
        filtered_df = df[df['Split'] == fold]
        index_list = filtered_df['Index'].tolist()
        labs = filtered_df['GT'].tolist()
        # for img_, label, weight, file in val_loader:
        for ind, index in enumerate(index_list):
            file='/home/raman/Work/Dataset/Images/'+str(index)+'.png'
            img_=homographic_transformation(file,index)

            transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0,0,0], std=[1,1,1]),
            ])
            img_ = transform(img_)
            pred = model(torch.stack([img_,]))
            
            cs = torch.matmul(torch.stack(outputs), pred[0].T)
            vals, inds = torch.topk(cs, 5)
            pred = round(sum(np.array(labels)[inds])/5)
            predictions.append(pred)

            #outputs[index] = output.detach()
            #labels[index] = labs[ind]
        
    return predictions

In [7]:
pred = get_visulaization("")

/home/raman/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/raman/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_381885/654284999.py:43: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/A